In [3]:
import dataset_utils
from dataset_utils import Index_read, Compute_adj_matrix, Compute_MST_from_adj
import numpy as np
import time
path = "./data/index_sift_learn_R64_L100_A1.2"
adj_dict = dataset_utils.Index_read(path)

print("--")
start = time.time()
sparse_adj_matrix = dataset_utils.Compute_adj_matrix(adj_dict,"fvec_datasets/sift_learn.fvecs")
end = time.time()
print("--Sparse adj matrix computed in", end - start)
# ngb = adj_dict[32][0]
# print(f"Distance between 32 and {ngb} from adj_matrix:",sparse_adj_matrix[32,ngb])
# input = dataset_utils.fvecs_read("fvec_datasets/sift_learn.fvecs")
# print(f"Distance between 32 and {ngb} from input:",np.linalg.norm(input[32,:] - input[ngb,:]))

start = time.time()
MST = dataset_utils.Compute_MST_from_adj(sparse_adj_matrix)
end = time.time()
print("MST computed in ", end - start)



print(f'non-zero edges {MST.size}')


#requires 74.5 GiB to dense
#print(MST.todense())




--
--Sparse adj matrix computed in 27.48433566093445
MST computed in  0.9148998260498047
  (0, 11970)	219.8567657470703
  (0, 28818)	260.5072021484375
  (1, 70310)	216.3839111328125
  (1, 56713)	270.920654296875
  (1, 2060)	261.5626220703125
  (2, 9783)	209.32749938964844
  (2, 25540)	235.0616912841797
  (2, 53790)	252.92884826660156
  (2, 44287)	255.39773559570312
  (3, 47660)	174.0287322998047
  (3, 98318)	178.1572265625
  (3, 78758)	230.63174438476562
  (4, 30815)	215.79852294921875
  (5, 7423)	224.61744689941406
  (5, 42939)	235.1637725830078
  (5, 85753)	240.9771728515625
  (5, 30513)	280.06427001953125
  (6, 4039)	264.9396057128906
  (6, 55356)	292.0873107910156
  (7, 45244)	208.3338623046875
  (7, 43614)	225.49722290039062
  (7, 18006)	252.30735778808594
  (7, 47539)	265.10186767578125
  (8, 25457)	192.2732391357422
  (9, 55382)	236.70022583007812
  :	:
  (99689, 99808)	166.27085876464844
  (99689, 99868)	162.35147094726562
  (99693, 99731)	200.0999755859375
  (99697, 99917)	125

<100000x100000 sparse matrix of type '<class 'numpy.float64'>'
	with 99765 stored elements in Compressed Sparse Row format>

In [10]:
import dataset_utils
from dataset_utils import Index_read, Compute_adj_matrix, Compute_MST_from_adj,fvecs_read
from scipy.spatial.distance import pdist, squareform
from scipy.sparse.csgraph import minimum_spanning_tree
import numpy as np
import time
path = "./data/index_Cancer_R64_L100_A1.2"
adj_dict = dataset_utils.Index_read(path)

print("--")
start = time.time()
sparse_adj_matrix = dataset_utils.Compute_adj_matrix(adj_dict,"fvec_datasets/Cancer.fvec")
end = time.time()
print("--Sparse adj matrix computed in", end - start)
# ngb = adj_dict[32][0]
# print(f"Distance between 32 and {ngb} from adj_matrix:",sparse_adj_matrix[32,ngb])
# input = dataset_utils.fvecs_read("fvec_datasets/sift_learn.fvecs")
# print(f"Distance between 32 and {ngb} from input:",np.linalg.norm(input[32,:] - input[ngb,:]))

start = time.time()
MST = dataset_utils.Compute_MST_from_adj(sparse_adj_matrix)
end = time.time()
print("MST computed in ", end - start)



print(f'non-zero edges {MST.size}')


print("trying with dense from sparse")

start = time.time()
MST = dataset_utils.Compute_MST_from_adj(sparse_adj_matrix.todense())
end = time.time()
MST

#inconsistence! Running with sparse matrix results in a florest instead of a tree

#requires 74.5 GiB to dense
#print(MST.todense())




--
--Sparse adj matrix computed in 0.05510759353637695
MST computed in  0.00099945068359375
non-zero edges 459
trying with dense from sparse


<699x699 sparse matrix of type '<class 'numpy.float64'>'
	with 698 stored elements in Compressed Sparse Row format>

In [7]:
import dataset_utils
from dataset_utils import Index_read, Compute_adj_matrix, Compute_MST_from_adj,fvecs_read
from scipy.spatial.distance import pdist, squareform
from scipy.sparse.csgraph import minimum_spanning_tree
import numpy as np
import time

input = dataset_utils.fvecs_read("fvec_datasets/Cancer.fvec")
start = time.time()
dists = squareform(pdist(input))
end = time.time()
print("--dense adj matrix computed in", end - start)
mst = minimum_spanning_tree(dists).toarray()
print("dense MST computed in ", end - start)
print(mst.shape)

--dense adj matrix computed in 0.004340410232543945
dense MST computed in  0.004340410232543945
(699, 699)
